### 筛选出评分最高的景点及其所在城市

In [1]:
import pandas as pd

file_path = 'merged_with_filenames.csv'
df = pd.read_csv(file_path)

# 移除 'File Name' 列中包含缺失值的行
df = df.dropna(subset=['File Name'])

# 检查评分列的数据类型并进行转换
df['评分'] = pd.to_numeric(df['评分'], errors='coerce')

# 移除 '评分' 列中包含缺失值的行
df = df.dropna(subset=['评分'])

# 每个 'File Name' 中评分最高的行
max_score_df = df.loc[df.groupby('File Name')['评分'].idxmax()]

output_path = '城市最佳景点游览原则.csv'
max_score_df.to_csv(output_path, index=False)


### 将景点名、城市名和城市指标合并

In [2]:
csv_file_path = '城市最佳景点游览原则.csv'
excel_file_path = 'C题最终版数据集.xlsx'

scenic_data = pd.read_csv(csv_file_path)

city_attributes_data = pd.read_excel(excel_file_path)

# 合并景点数据和城市属性数据
merged_data = pd.merge(scenic_data, city_attributes_data, left_on='File Name', right_on='来源城市')

output_path = '各城市指标最终版.csv'
merged_data.to_csv(output_path, index=False)


### 使用层次分析法对每个评分最高的景点所在城市评分

In [3]:
import ahpy
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

comparisons = {
    ('AQI', '绿化覆盖率 (%)'): 1/3,
    ('AQI', '废水处理率 (%)'): 1/3,
    ('AQI', '废气处理率 (%)'): 1/3,
    ('AQI', '垃圾分类处理率 (%)'): 1/3,
    ('AQI', '历史遗迹数量'): 1/5,
    ('AQI', '博物馆数量'): 1/5,
    ('AQI', '文化活动频次'): 1/5,
    ('AQI', '文化设施数量'): 1/7,
    ('AQI', '公共交通覆盖率 (%)'): 1/7,
    ('AQI', '线路密度 (km/km²)'): 1/7,
    ('AQI', '高速公路里程 (km)'): 1/9,
    ('AQI', '机场航班数量'): 1/9,
    ('AQI', '年平均气温 (℃)'): 1/9,
    ('AQI', '年降水量 (mm)'): 1/9,
    ('AQI', '适宜旅游天数'): 1/9,
    ('AQI', '空气湿度 (%)'): 1/9,
    ('AQI', '餐馆数量'): 1/9,
    ('AQI', '特色美食数量'): 1/9,
    ('AQI', '美食活动频次'): 1/9,
    ('绿化覆盖率 (%)', '废水处理率 (%)'): 1,
    ('绿化覆盖率 (%)', '废气处理率 (%)'): 1,
    ('绿化覆盖率 (%)', '垃圾分类处理率 (%)'): 1,
    ('绿化覆盖率 (%)', '历史遗迹数量'): 1/3,
    ('绿化覆盖率 (%)', '博物馆数量'): 1/3,
    ('绿化覆盖率 (%)', '文化活动频次'): 1/3,
    ('绿化覆盖率 (%)', '文化设施数量'): 1/5,
    ('绿化覆盖率 (%)', '公共交通覆盖率 (%)'): 1/5,
    ('绿化覆盖率 (%)', '线路密度 (km/km²)'): 1/5,
    ('绿化覆盖率 (%)', '高速公路里程 (km)'): 1/7,
    ('绿化覆盖率 (%)', '机场航班数量'): 1/7,
    ('绿化覆盖率 (%)', '年平均气温 (℃)'): 1/7,
    ('绿化覆盖率 (%)', '年降水量 (mm)'): 1/7,
    ('绿化覆盖率 (%)', '适宜旅游天数'): 1/7,
    ('绿化覆盖率 (%)', '空气湿度 (%)'): 1/7,
    ('绿化覆盖率 (%)', '餐馆数量'): 1/7,
    ('绿化覆盖率 (%)', '特色美食数量'): 1/7,
    ('绿化覆盖率 (%)', '美食活动频次'): 1/7,
    ('废水处理率 (%)', '废气处理率 (%)'): 1,
    ('废水处理率 (%)', '垃圾分类处理率 (%)'): 1,
    ('废水处理率 (%)', '历史遗迹数量'): 1/3,
    ('废水处理率 (%)', '博物馆数量'): 1/3,
    ('废水处理率 (%)', '文化活动频次'): 1/3,
    ('废水处理率 (%)', '文化设施数量'): 1/5,
    ('废水处理率 (%)', '公共交通覆盖率 (%)'): 1/5,
    ('废水处理率 (%)', '线路密度 (km/km²)'): 1/5,
    ('废水处理率 (%)', '高速公路里程 (km)'): 1/7,
    ('废水处理率 (%)', '机场航班数量'): 1/7,
    ('废水处理率 (%)', '年平均气温 (℃)'): 1/7,
    ('废水处理率 (%)', '年降水量 (mm)'): 1/7,
    ('废水处理率 (%)', '适宜旅游天数'): 1/7,
    ('废水处理率 (%)', '空气湿度 (%)'): 1/7,
    ('废水处理率 (%)', '餐馆数量'): 1/7,
    ('废水处理率 (%)', '特色美食数量'): 1/7,
    ('废水处理率 (%)', '美食活动频次'): 1/7,
    ('废气处理率 (%)', '垃圾分类处理率 (%)'): 1,
    ('废气处理率 (%)', '历史遗迹数量'): 1/3,
    ('废气处理率 (%)', '博物馆数量'): 1/3,
    ('废气处理率 (%)', '文化活动频次'): 1/3,
    ('废气处理率 (%)', '文化设施数量'): 1/5,
    ('废气处理率 (%)', '公共交通覆盖率 (%)'): 1/5,
    ('废气处理率 (%)', '线路密度 (km/km²)'): 1/5,
    ('废气处理率 (%)', '高速公路里程 (km)'): 1/7,
    ('废气处理率 (%)', '机场航班数量'): 1/7,
    ('废气处理率 (%)', '年平均气温 (℃)'): 1/7,
    ('废气处理率 (%)', '年降水量 (mm)'): 1/7,
    ('废气处理率 (%)', '适宜旅游天数'): 1/7,
    ('废气处理率 (%)', '空气湿度 (%)'): 1/7,
    ('废气处理率 (%)', '餐馆数量'): 1/7,
    ('废气处理率 (%)', '特色美食数量'): 1/7,
    ('废气处理率 (%)', '美食活动频次'): 1/7,
    ('垃圾分类处理率 (%)', '历史遗迹数量'): 1/3,
    ('垃圾分类处理率 (%)', '博物馆数量'): 1/3,
    ('垃圾分类处理率 (%)', '文化活动频次'): 1/3,
    ('垃圾分类处理率 (%)', '文化设施数量'): 1/5,
    ('垃圾分类处理率 (%)', '公共交通覆盖率 (%)'): 1/5,
    ('垃圾分类处理率 (%)', '线路密度 (km/km²)'): 1/5,
    ('垃圾分类处理率 (%)', '高速公路里程 (km)'): 1/7,
    ('垃圾分类处理率 (%)', '机场航班数量'): 1/7,
    ('垃圾分类处理率 (%)', '年平均气温 (℃)'): 1/7,
    ('垃圾分类处理率 (%)', '年降水量 (mm)'): 1/7,
    ('垃圾分类处理率 (%)', '适宜旅游天数'): 1/7,
    ('垃圾分类处理率 (%)', '空气湿度 (%)'): 1/7,
    ('垃圾分类处理率 (%)', '餐馆数量'): 1/7,
    ('垃圾分类处理率 (%)', '特色美食数量'): 1/7,
    ('垃圾分类处理率 (%)', '美食活动频次'): 1/7,
    ('历史遗迹数量', '博物馆数量'): 1,
    ('历史遗迹数量', '文化活动频次'): 1,
    ('历史遗迹数量', '文化设施数量'): 1/3,
    ('历史遗迹数量', '公共交通覆盖率 (%)'): 1/3,
    ('历史遗迹数量', '线路密度 (km/km²)'): 1/3,
    ('历史遗迹数量', '高速公路里程 (km)'): 1/5,
    ('历史遗迹数量', '机场航班数量'): 1/5,
    ('历史遗迹数量', '年平均气温 (℃)'): 1/5,
    ('历史遗迹数量', '年降水量 (mm)'): 1/5,
    ('历史遗迹数量', '适宜旅游天数'): 1/5,
    ('历史遗迹数量', '空气湿度 (%)'): 1/5,
    ('历史遗迹数量', '餐馆数量'): 1/5,
    ('历史遗迹数量', '特色美食数量'): 1/5,
    ('历史遗迹数量', '美食活动频次'): 1/5,
    ('博物馆数量', '文化活动频次'): 1,
    ('博物馆数量', '文化设施数量'): 1/3,
    ('博物馆数量', '公共交通覆盖率 (%)'): 1/3,
    ('博物馆数量', '线路密度 (km/km²)'): 1/3,
    ('博物馆数量', '高速公路里程 (km)'): 1/5,
    ('博物馆数量', '机场航班数量'): 1/5,
    ('博物馆数量', '年平均气温 (℃)'): 1/5,
    ('博物馆数量', '年降水量 (mm)'): 1/5,
    ('博物馆数量', '适宜旅游天数'): 1/5,
    ('博物馆数量', '空气湿度 (%)'): 1/5,
    ('博物馆数量', '餐馆数量'): 1/5,
    ('博物馆数量', '特色美食数量'): 1/5,
    ('博物馆数量', '美食活动频次'): 1/5,
    ('文化活动频次', '文化设施数量'): 1/3,
    ('文化活动频次', '公共交通覆盖率 (%)'): 1/3,
    ('文化活动频次', '线路密度 (km/km²)'): 1/3,
    ('文化活动频次', '高速公路里程 (km)'): 1/5,
    ('文化活动频次', '机场航班数量'): 1/5,
    ('文化活动频次', '年平均气温 (℃)'): 1/5,
    ('文化活动频次', '年降水量 (mm)'): 1/5,
    ('文化活动频次', '适宜旅游天数'): 1/5,
    ('文化活动频次', '空气湿度 (%)'): 1/5,
    ('文化活动频次', '餐馆数量'): 1/5,
    ('文化活动频次', '特色美食数量'): 1/5,
    ('文化活动频次', '美食活动频次'): 1/5,
    ('文化设施数量', '公共交通覆盖率 (%)'): 1,
    ('文化设施数量', '线路密度 (km/km²)'): 1,
    ('文化设施数量', '高速公路里程 (km)'): 1/3,
    ('文化设施数量', '机场航班数量'): 1/3,
    ('文化设施数量', '年平均气温 (℃)'): 1/3,
    ('文化设施数量', '年降水量 (mm)'): 1/3,
    ('文化设施数量', '适宜旅游天数'): 1/3,
    ('文化设施数量', '空气湿度 (%)'): 1/3,
    ('文化设施数量', '餐馆数量'): 1/3,
    ('文化设施数量', '特色美食数量'): 1/3,
    ('文化设施数量', '美食活动频次'): 1/3,
    ('公共交通覆盖率 (%)', '线路密度 (km/km²)'): 1,
    ('公共交通覆盖率 (%)', '高速公路里程 (km)'): 1/3,
    ('公共交通覆盖率 (%)', '机场航班数量'): 1/3,
    ('公共交通覆盖率 (%)', '年平均气温 (℃)'): 1/3,
    ('公共交通覆盖率 (%)', '年降水量 (mm)'): 1/3,
    ('公共交通覆盖率 (%)', '适宜旅游天数'): 1/3,
    ('公共交通覆盖率 (%)', '空气湿度 (%)'): 1/3,
    ('公共交通覆盖率 (%)', '餐馆数量'): 1/3,
    ('公共交通覆盖率 (%)', '特色美食数量'): 1/3,
    ('公共交通覆盖率 (%)', '美食活动频次'): 1/3,
    ('线路密度 (km/km²)', '高速公路里程 (km)'): 1/3,
    ('线路密度 (km/km²)', '机场航班数量'): 1/3,
    ('线路密度 (km/km²)', '年平均气温 (℃)'): 1/3,
    ('线路密度 (km/km²)', '年降水量 (mm)'): 1/3,
    ('线路密度 (km/km²)', '适宜旅游天数'): 1/3,
    ('线路密度 (km/km²)', '空气湿度 (%)'): 1/3,
    ('线路密度 (km/km²)', '餐馆数量'): 1/3,
    ('线路密度 (km/km²)', '特色美食数量'): 1/3,
    ('线路密度 (km/km²)', '美食活动频次'): 1/3,
    ('高速公路里程 (km)', '机场航班数量'): 1,
    ('高速公路里程 (km)', '年平均气温 (℃)'): 1,
    ('高速公路里程 (km)', '年降水量 (mm)'): 1,
    ('高速公路里程 (km)', '适宜旅游天数'): 1,
    ('高速公路里程 (km)', '空气湿度 (%)'): 1,
    ('高速公路里程 (km)', '餐馆数量'): 1,
    ('高速公路里程 (km)', '特色美食数量'): 1,
    ('高速公路里程 (km)', '美食活动频次'): 1,
    ('机场航班数量', '年平均气温 (℃)'): 1,
    ('机场航班数量', '年降水量 (mm)'): 1,
    ('机场航班数量', '适宜旅游天数'): 1,
    ('机场航班数量', '空气湿度 (%)'): 1,
    ('机场航班数量', '餐馆数量'): 1,
    ('机场航班数量', '特色美食数量'): 1,
    ('机场航班数量', '美食活动频次'): 1,
    ('年平均气温 (℃)', '年降水量 (mm)'): 1,
    ('年平均气温 (℃)', '适宜旅游天数'): 1,
    ('年平均气温 (℃)', '空气湿度 (%)'): 1,
    ('年平均气温 (℃)', '餐馆数量'): 1,
    ('年平均气温 (℃)', '特色美食数量'): 1,
    ('年平均气温 (℃)', '美食活动频次'): 1,
    ('年降水量 (mm)', '适宜旅游天数'): 1,
    ('年降水量 (mm)', '空气湿度 (%)'): 1,
    ('年降水量 (mm)', '餐馆数量'): 1,
    ('年降水量 (mm)', '特色美食数量'): 1,
    ('年降水量 (mm)', '美食活动频次'): 1,
    ('适宜旅游天数', '空气湿度 (%)'): 1,
    ('适宜旅游天数', '餐馆数量'): 1,
    ('适宜旅游天数', '特色美食数量'): 1,
    ('适宜旅游天数', '美食活动频次'): 1,
    ('空气湿度 (%)', '餐馆数量'): 1,
    ('空气湿度 (%)', '特色美食数量'): 1,
    ('空气湿度 (%)', '美食活动频次'): 1,
    ('餐馆数量', '特色美食数量'): 1,
    ('餐馆数量', '美食活动频次'): 1,
    ('特色美食数量', '美食活动频次'): 1
    }


criteria = ahpy.Compare('Criteria', comparisons, precision=4)

weights = criteria.target_weights

for criterion, weight in weights.items():
    print(f'{criterion}: {weight:.4f}')

weights_df = pd.DataFrame.from_dict(weights, orient='index', columns=['weight'])

city_indicators_path = '各城市指标最终版.csv'
city_indicators_data = pd.read_csv(city_indicators_path)


attributes = [
'AQI', '绿化覆盖率 (%)', '废水处理率 (%)', '废气处理率 (%)', '垃圾分类处理率 (%)',
'历史遗迹数量', '博物馆数量', '文化活动频次', '文化设施数量', '公共交通覆盖率 (%)',
'线路密度 (km/km²)', '高速公路里程 (km)', '机场航班数量', '年平均气温 (℃)',
'年降水量 (mm)', '适宜旅游天数', '空气湿度 (%)', '餐馆数量', '特色美食数量', '美食活动频次'
]
scaler = StandardScaler()
city_indicators_data[attributes] = scaler.fit_transform(city_indicators_data[attributes])


city_indicators_data['综合评分'] = city_indicators_data[attributes].dot(weights_df['weight'])


top_50_cities = city_indicators_data.sort_values(by='综合评分', ascending=False).head(50)


output_path = '最令外国游客向往的50个城市（AHP）最终版.csv'
top_50_cities.to_csv(output_path, index=False)

高速公路里程 (km): 0.0860
机场航班数量: 0.0860
年平均气温 (℃): 0.0860
年降水量 (mm): 0.0860
适宜旅游天数: 0.0860
空气湿度 (%): 0.0860
餐馆数量: 0.0860
特色美食数量: 0.0860
美食活动频次: 0.0860
文化设施数量: 0.0392
公共交通覆盖率 (%): 0.0392
线路密度 (km/km²): 0.0392
历史遗迹数量: 0.0199
博物馆数量: 0.0199
文化活动频次: 0.0199
绿化覆盖率 (%): 0.0104
废水处理率 (%): 0.0104
废气处理率 (%): 0.0104
垃圾分类处理率 (%): 0.0104
AQI: 0.0065
